<a href="https://colab.research.google.com/github/prisqia/EDA-Olympics-2021/blob/main/EDA_Olympics_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connect To Kaggle

In [2]:
! pip install kaggle

In [3]:
! mkdir ~/.kaggle

In [4]:
! cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

# Download Kaggle Dataset


In this EDA we used 2021 Olympics in Tokyo Dataset

In [6]:
! kaggle datasets download arjunprasadsarkhel/2021-olympics-in-tokyo

  0% 0.00/348k [00:00<?, ?B/s]
100% 348k/348k [00:00<00:00, 52.1MB/s]


In [11]:
! mkdir Tokyo_Olympics

In [12]:
! unzip 2021-olympics-in-tokyo.zip -d Tokyo_Olympics

Archive:  2021-olympics-in-tokyo.zip
  inflating: Tokyo_Olympics/Athletes.xlsx  
  inflating: Tokyo_Olympics/Coaches.xlsx  
  inflating: Tokyo_Olympics/EntriesGender.xlsx  
  inflating: Tokyo_Olympics/Medals.xlsx  
  inflating: Tokyo_Olympics/Teams.xlsx  


#Import Library

In [22]:
import pandas as pd
import numpy as np
from scipy.stats import trim_mean

# Create a Path from Data Folder

In [19]:
path_to_data = '/content/Tokyo_Olympics/'

In [21]:
df_athletes = pd.read_excel(path_to_data + 'Athletes.xlsx')
df_gender = pd.read_excel(path_to_data + 'EntriesGender.xlsx')
df_medals = pd.read_excel(path_to_data + 'Medals.xlsx')
df_teams = pd.read_excel(path_to_data + 'Teams.xlsx')

#EDA

In [34]:
df_medals

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,39,41,33,113,1
1,2,People's Republic of China,38,32,18,88,2
2,3,Japan,27,14,17,58,5
3,4,Great Britain,22,21,22,65,4
4,5,ROC,20,28,23,71,3
...,...,...,...,...,...,...,...
88,86,Ghana,0,0,1,1,77
89,86,Grenada,0,0,1,1,77
90,86,Kuwait,0,0,1,1,77
91,86,Republic of Moldova,0,0,1,1,77


In [47]:
df_medals.isna().sum()

Rank             0
Team/NOC         0
Gold             0
Silver           0
Bronze           0
Total            0
Rank by Total    0
dtype: int64

In [27]:
df_medals['Total'].mean()

11.612903225806452

In [28]:
np.average(df_medals["Total"], weights=df_medals["Gold"])

46.832352941176474

In [29]:
trim_mean(df_medals["Total"], 0.1)

6.8933333333333335

In [32]:
from statistics import variance, stdev

variance(df_medals["Gold"])

49.315100514259

In [33]:
stdev(df_medals["Gold"])

7.022471111671376

In [37]:
def weighted_median(df, median_col, weight_col) :
  df_sorted = df.sort_values(median_col)
  cumsum = df_sorted[weight_col].cumsum()
  cutoff = df_sorted[weight_col].sum() / 2
  return df_sorted[cumsum >= cutoff][median_col].iloc[0]

weighted_median(df_medals,"Total", "Gold")


40

In [46]:
q3 = np.percentile(df_medals["Total"], 75)
q3

11.0